In [1]:
import os
import sys

# Obtener el directorio actual de la notebook
notebook_dir = os.getcwd()

# Obtener el directorio padre
parent_dir = os.path.abspath(os.path.join(notebook_dir, ".."))

# Agregar el directorio padre al path
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [2]:
from libs.config.config_variables import (
    MAX_WORKERS,
    TIMEOUT_API_REQUEST,
    CATALOG_CSV_DIR,
)

from modules.catalog_scraper.seismic_scraper import SeismicScraper
from modules.downloader.records_downloader import SeismicDownloader

# 0. Configuración básica de catálogos

In [3]:
# Límites geográficos (Perú y alrededores)
MIN_LAT, MAX_LAT = -20.0, 0.0
MIN_LON, MAX_LON = -85.0, -68.0

# Año de inicio del scraping
START_YEAR = 1400

# 1. Ejecución de scraping de catálogos


## 1.1 Carga de datos históricos

In [4]:
# Registrar los catálogos csv
historical_scraper = SeismicScraper(
    max_workers=MAX_WORKERS, timeout=TIMEOUT_API_REQUEST, csv_base_path=CATALOG_CSV_DIR
)
results = historical_scraper.scrape_events_from_csv(
    catalogs=["gcmt", "isc-gem", "sara"],
    # start_year=START_YEAR,
    # min_lat=MIN_LAT,
    # max_lat=MAX_LAT,
    # min_lon=MIN_LON,
    # max_lon=MAX_LON,
)


2025-11-23 12:38:18,247 [INFO] modules.catalog_scraper.seismic_scraper: Starting execution of SeismicScraper.scrape_events_from_csv
2025-11-23 12:38:18,247 [INFO] modules.catalog_scraper.seismic_scraper: Encontrados 1 archivos CSV para GCMT
2025-11-23 12:38:18,247 [INFO] modules.catalog_scraper.seismic_scraper: Encontrados 2 archivos CSV para ISC-GEM
2025-11-23 12:38:18,253 [INFO] modules.catalog_scraper.seismic_scraper: Encontrados 2 archivos CSV para SARA
2025-11-23 12:38:18,254 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando carga desde CSV: 5 archivos con 40 workers
2025-11-23 12:38:18,254 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando carga desde CSV: GCMT - gcmt_data.csv
2025-11-23 12:38:18,272 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando carga desde CSV: ISC-GEM - isc-gem-cat.csv
2025-11-23 12:38:18,272 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando carga desde CSV: ISC-GEM - isc-gem-suppl.csv
2025-11-23 12:38:18,272 [INFO] modules.ca

In [5]:
historical_scraper.metadata

+-----------------+--------+--------------+---------+
| Total procesado | Nuevos | Actualizados | Errores |
+-----------------+--------+--------------+---------+
|     236356      | 64879  |    171477    |    1    |
+-----------------+--------+--------------+---------+


## 1.2 Actualización de datos recientes

In [ ]:
recent_data_scraper = SeismicScraper(
    max_workers=MAX_WORKERS, timeout=TIMEOUT_API_REQUEST
)

results = recent_data_scraper.scrape_events(
    start_year=2000,
    min_lat=MIN_LAT,
    max_lat=MAX_LAT,
    min_lon=MIN_LON,
    max_lon=MAX_LON,
    catalogs=["usgs", "isc", "igp"],
)

2025-11-23 12:38:35,521 [INFO] modules.catalog_scraper.seismic_scraper: Starting execution of SeismicScraper.scrape_events
2025-11-23 12:38:35,521 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: 78 tareas con 40 workers
2025-11-23 12:38:35,521 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 2000
2025-11-23 12:38:35,547 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 2001
2025-11-23 12:38:35,577 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 2002
2025-11-23 12:38:35,579 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 2003
2025-11-23 12:38:35,608 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 2004
2025-11-23 12:38:35,611 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 2005
2025-11-23 12:38:35,640 [INFO] modules.catalog_scraper.seismic_scraper: Iniciando scraping: USGS - 2006
2025-11-23 12:38:35,640 [INFO] m

In [7]:
recent_data_scraper.metadata

+-----------------+--------+--------------+---------+
| Total procesado | Nuevos | Actualizados | Errores |
+-----------------+--------+--------------+---------+
|      15214      |  927   |    14287     |   15    |
+-----------------+--------+--------------+---------+


## 1.3 Descarga de registros acelerométricos

In [8]:
NUM_EVENTS = 100
MIN_MAGNITUDE = 4.0
CATALOGS = ["igp"]  # ["igp", "usgs", "emsc"] cuando estén disponibles

In [ ]:
downloader = SeismicDownloader()

for catalog in CATALOGS:
    results = downloader.process_events(
        catalog=catalog,
        num_events_per_catalog=NUM_EVENTS,
        max_workers=2,
        parallel_stations=MAX_WORKERS,
        min_magnitude=MIN_MAGNITUDE,
    )
    print(f"Procesando catálogo: {catalog}")

2025-11-23 12:38:49,052 [INFO] libs.database.seismic_records: Conexión establecida a c:\Users\joel.alarcon\Desktop\_code\gura_forge\data\database\gure_forge.db
2025-11-23 12:38:49,055 [INFO] modules.downloader.records_downloader: Starting execution of SeismicDownloader.process_events


TypeError: SeismicDownloader.process_events() got an unexpected keyword argument 'catalogs'

In [ ]:
downloader.metadata